In [1]:
using Pkg
Pkg.activate(".")

using Fairness
using MLJBase, MLJModels # Alternatively you can do "using MLJ" if MLJ if you have installed it

 Activating environment at `~/Desktop/Fairness/Project.toml`


### Loading toy data

In [2]:
X, y, ŷ = @load_toydata;

In [3]:
X

,Sex,Ethnicity,Highest degree,Job type
,Cat…,Cat…,Cat…,Cat…
1,M,Native,H. school,Board
2,M,Native,Univ.,Board
3,M,Native,H. school,Board
4,M,Non-nat.,H. school,Healthcare
5,M,Non-nat.,Univ.,Healthcare
6,F,Non-nat.,Univ.,Education
7,F,Native,H. school,Education
8,F,Native,None,Healthcare
9,F,Non-nat.,Univ.,Education


In [4]:
y, ŷ

(CategoricalArrays.CategoricalValue{Int64,UInt32}[1, 1, 1, 1, 0, 0, 0, 1, 0, 1], CategoricalArrays.CategoricalValue{Int64,UInt32}[1, 0, 1, 0, 1, 0, 1, 0, 1, 0])

# Loading the Classifier and Wrapping it

## ReweighingSampling Algorithm

In [5]:
model = ConstantClassifier()

wrappedModel = ReweighingSamplingWrapper(classifier=model, grp=:Sex)

ReweighingSamplingWrapper(
    grp = :Sex,
    classifier = ConstantClassifier(),
    factor = 1.0,
    rng = Random._GLOBAL_RNG()) @268

### Using MLJ for packaging data with the Machine Learning model (Wrapped by fairness algorithm)

In [6]:
mach = machine(wrappedModel, X, y)

Machine{ReweighingSamplingWrapper{ConstantClassifier}} @447 trained 0 times.
  args: 
    1:	Source @010 ⏎ `Table{Union{AbstractArray{Multiclass{2},1}, AbstractArray{Multiclass{3},1}}}`
    2:	Source @029 ⏎ `AbstractArray{Multiclass{2},1}`


### Fitting the Wrapped Model on the packaged data.

Everything will be handled underneath the hood. The reweighing algorithm will be fitted first to generate weights for all comninations of classes and outcomes. Then these weights are used to sample the training data and then classifier is trained....**And all this is done automatically by Fairness.jl. You simply have to choose the wrapper of your choice!!**

In [7]:
fit!(mach)

┌ Info: Training Machine{ReweighingSamplingWrapper{ConstantClassifier}} @447.
└ @ MLJBase /home/ashryaagr/.julia/packages/MLJBase/CcEkh/src/machines.jl:317
┌ Info: Training Machine{ConstantClassifier} @170.
└ @ MLJBase /home/ashryaagr/.julia/packages/MLJBase/CcEkh/src/machines.jl:317


Machine{ReweighingSamplingWrapper{ConstantClassifier}} @447 trained 1 time.
  args: 
    1:	Source @010 ⏎ `Table{Union{AbstractArray{Multiclass{2},1}, AbstractArray{Multiclass{3},1}}}`
    2:	Source @029 ⏎ `AbstractArray{Multiclass{2},1}`


Prediction is also handled underneath the hood for all sorts of algorithms.

Fairness.jl leverages MLJ to make it really easy to do fitting and prediction.
Internals: MLJ creates a node for the end classifier and automatically calls predict on that node. So I didn't have to eplicitly code a predict function

In [8]:
predict(mach, X[1:5, :])

5-element UnivariateFiniteArray{Multiclass{2},Int64,UInt32,Float64,1}:
 UnivariateFinite{Multiclass{2}}(0=>0.5, 1=>0.5)
 UnivariateFinite{Multiclass{2}}(0=>0.5, 1=>0.5)
 UnivariateFinite{Multiclass{2}}(0=>0.5, 1=>0.5)
 UnivariateFinite{Multiclass{2}}(0=>0.5, 1=>0.5)
 UnivariateFinite{Multiclass{2}}(0=>0.5, 1=>0.5)

## Equalized Odds Postprocessing Algorithm

Again, you have to simply load the classifier model and wrap it with fairness algorithm of your choice!

In [9]:
model2 = ConstantClassifier()

wrappedModel2 = EqOddsWrapper(classifier=model2, grp=:Sex)

EqOddsWrapper(
    grp = :Sex,
    classifier = ConstantClassifier()) @366

### Package data with wrapped model  and then fit the model on that data

In [10]:
mach2 = machine(wrappedModel2, X, y)

fit!(mach2)

┌ Info: Training Machine{EqOddsWrapper{ConstantClassifier}} @115.
└ @ MLJBase /home/ashryaagr/.julia/packages/MLJBase/CcEkh/src/machines.jl:317
┌ Info: Training Machine{ConstantClassifier} @687.
└ @ MLJBase /home/ashryaagr/.julia/packages/MLJBase/CcEkh/src/machines.jl:317


Machine{EqOddsWrapper{ConstantClassifier}} @115 trained 1 time.
  args: 
    1:	Source @698 ⏎ `Table{Union{AbstractArray{Multiclass{2},1}, AbstractArray{Multiclass{3},1}}}`
    2:	Source @200 ⏎ `AbstractArray{Multiclass{2},1}`


### You can also see the training results i.e. the fitted params

What you shall see as output below is the fitting result for:
- Equalized Odds : It is in form of **[sp2n, sn2p, op2n, on2p]**. s denotes priviledged group and p2n denoted conversion from Positive to Negative
- Constant Classifier

In [11]:
mach2.fitresult

3-element Array{Any,1}:
 [[0.0, 0.0], [1.0, 1.0]]
 (CategoricalArrays.CategoricalValue{Int64,UInt32}[0, 1], [0.4 0.6])
 [0, 1]

### Predict

In [12]:
predict(mach2, X[5:10, :])

6-element CategoricalArrays.CategoricalArray{Int64,1,UInt32}:
 1
 1
 1
 1
 1
 1